<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/XinferenceLocalDeployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Xorbits推断

在这个演示笔记本中，我们将展示如何使用Xorbits推断（简称Xinference）来在三个步骤中部署本地LLM。

在本示例中，我们将使用Llama 2聊天模型的GGML格式，但是代码应该很容易地转移到Xinference支持的所有LLM聊天模型上。以下是一些示例：

| 名称          | 类型             | 语言 | 格式   | 大小（十亿） | 量化                                |
|---------------|------------------|------|--------|--------------|-------------------------------------|
| llama-2-chat  | RLHF模型         | en   | ggmlv3 | 7, 13, 70    | 'q2_K', 'q3_K_L', ... , 'q6_K', 'q8_0' |
| chatglm       | SFT模型          | en, zh | ggmlv3 | 6            | 'q4_0', 'q4_1', 'q5_0', 'q5_1', 'q8_0' |
| chatglm2      | SFT模型          | en, zh | ggmlv3 | 6            | 'q4_0', 'q4_1', 'q5_0', 'q5_1', 'q8_0' |
| wizardlm-v1.0 | SFT模型          | en   | ggmlv3 | 7, 13, 33    | 'q2_K', 'q3_K_L', ... , 'q6_K', 'q8_0' |
| wizardlm-v1.1 | SFT模型          | en   | ggmlv3 | 13           | 'q2_K', 'q3_K_L', ... , 'q6_K', 'q8_0' |
| vicuna-v1.3   | SFT模型          | en   | ggmlv3 | 7, 13        | 'q2_K', 'q3_K_L', ... , 'q6_K', 'q8_0' |

最新的支持模型完整列表可以在Xorbits推断的[官方GitHub页面](https://github.com/xorbitsai/inference/blob/main/README.md)中找到。


## <span style="font-size: xx-large;;">🤖  </span> 安装 Xinference

i. 在终端窗口中运行 `pip install "xinference[all]"`。

ii. 安装完成后，重新启动这个jupyter笔记本。

iii. 在新的终端窗口中运行 `xinference`。

iv. 你应该会看到类似以下输出：

```
INFO:xinference:Xinference successfully started. Endpoint: http://127.0.0.1:9997
INFO:xinference.core.service:Worker 127.0.0.1:21561 has been added successfully
INFO:xinference.deploy.worker:Xinference worker successfully started.
```

v. 在端点描述中，找到冒号后的端口号。在上面的例子中，端口号是 `9997`。

vi. 使用以下单元格设置端口号：


In [ ]:
%pip install llama-index-llms-xinference

In [ ]:
端口 = 9997  # 替换为您的端点端口号

## <span style="font-size: xx-large;;">🚀  </span> 启动本地模型

在这一步中，我们将从`llama_index`导入相关的库。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

In [ ]:
# 如果无法导入Xinference，则可能需要重新启动jupyter笔记本from llama_index.core import SummaryIndexfrom llama_index.core import (    TreeIndex,    VectorStoreIndex,    KeywordTableIndex,    KnowledgeGraphIndex,    SimpleDirectoryReader,)from llama_index.llms.xinference import Xinferencefrom xinference.client import RESTfulClientfrom IPython.display import Markdown, display

然后，我们启动一个模型并使用它。这样可以让我们在后续步骤中将模型连接到文档和查询。

请随意更改参数以获得更好的性能！为了达到最佳结果，建议使用大小超过13B的模型。也就是说，7B模型对于这个简短的演示已经足够了。

以下是GGML格式中Llama 2聊天模型的一些更多参数选项，按照占用空间最少到资源消耗最多但性能高的顺序列出。

<span style="font-weight: bold; ;">模型大小（以十亿为单位）：</span>

`7`, `13`, `70`

<span style="font-weight: bold; ;">7B和13B模型的量化选项：</span>

`q2_K`, `q3_K_L`, `q3_K_M`, `q3_K_S`, `q4_0`, `q4_1`, `q4_K_M`, `q4_K_S`, `q5_0`, `q5_1`, `q5_K_M`, `q5_K_S`, `q6_K`, `q8_0`

<span style="font-weight: bold; ;">70B模型的量化选项：</span>

`q4_0`


In [ ]:
# 定义一个客户端，用于向xinference发送命令client = RESTfulClient(f"http://localhost:{port}")# 下载并启动一个模型，第一次可能需要一段时间model_uid = client.launch_model(    model_name="llama-2-chat",    model_size_in_billions=7,    model_format="ggmlv3",    quantization="q2_K",)# 初始化Xinference对象以使用LLMllm = Xinference(    endpoint=f"http://localhost:{port}",    model_uid=model_uid,    temperature=0.0,    max_tokens=512,)

## <span style="font-size: xx-large;;">🕺  </span> 索引数据...并聊天！

在这一步中，我们将模型和数据结合起来创建一个查询引擎。查询引擎可以用作聊天机器人，根据给定的数据回答我们的查询。

我们将使用`VetorStoreIndex`，因为它相对较快。也就是说，可以随意更改索引以获得不同的体验。以下是从上一步已经导入的一些可用索引：

`ListIndex`、`TreeIndex`、`VetorStoreIndex`、`KeywordTableIndex`、`KnowledgeGraphIndex`

要更改索引，只需在下面的代码中将`VetorStoreIndex`替换为另一个索引即可。

所有可用索引的最新完整列表可以在 Llama Index 的[官方文档](https://gpt-index.readthedocs.io/en/latest/core_modules/data_modules/index/modules.html)中找到。


In [ ]:
# 从数据中创建索引documents = SimpleDirectoryReader("../data/paul_graham").load_data()# 在下面的代码中更改索引名称index = VectorStoreIndex.from_documents(documents=documents)# 创建查询引擎query_engine = index.as_query_engine(llm=llm)

我们可以在询问问题之前通过`Xinference`对象直接设置温度和最大答案长度（以标记为单位）。这样可以在不需要每次重建查询引擎的情况下为不同的问题更改参数。

`temperature`是一个介于0和1之间的数字，用于控制响应的随机性。较高的值会增加创造力，但可能导致跑题的回复。将其设置为零可以确保每次都得到相同的回复。

`max_tokens`是一个整数，用于设置响应长度的上限。如果答案似乎被截断，可以增加这个值，但要注意太长的回复可能会超出上下文窗口并导致错误。


In [ ]:
# 可选地，更新温度和最大答案长度（以标记为单位）llm.__dict__.update({"temperature": 0.0})llm.__dict__.update({"max_tokens": 2048})# 提出一个问题并显示答案question = "作者在Y Combinator结束后做了什么？"response = query_engine.query(question)display(Markdown(f"<b>{response}</b>"))